# Übung 2 vom 18.10.2022

### Exercise 1

Entwerfen Sie einen Algorithmus zur gerechten Aufteilung eines Kuchens unter 3 Personen. Beschreiben Sie den Algorithmus in geeigneter Weise, zum Beispiel in Pseudocode
oder als Nassi-Shneidermann Diagramm. Auch bebilderter Fließtext kann sinnvoll sein.

In [7]:
from dataclasses import dataclass, field
import random as random

@dataclass
class Person:
    name: str
    estimate_percent: float = 0
    percent: float = 0
    cake: float = 0
    position: int = field(init=False, default=0, repr=False)

    def __str__(self) -> str:
        return f"{self.position}. Aufteilung: {self.name} schätzt {round(self.estimate_percent*100,2)}% und bekommt {round(self.cake,2)} ({round((self.cake/Group.cake*100),2)}%) Kuchen ({round(self.percent*100,2)}%, diff {round((self.percent-self.estimate_percent)*100,2)})"

@dataclass
class Group:
    name: str
    members: list[Person]
    cake: float = 100

    def __init__(self, name: str, members: list[Person]) -> None:
        self.name = name
        self.members = members

    def __str__(self) -> str:
        # sort members by estimate_percent
        members_sorted = sorted(self.members, key=lambda member: member.position)
        mermbers_str = "\n  - ".join([str(member) for member in members_sorted])
        return f"Gruppe: {self.name}\n- Mitglieder: {len(self.members)}\n  - {mermbers_str}"


In [8]:
members = [
    Person("Alice"),
    Person("Bob"),
    Person("Charlie"),
    # Person("David"),
    # Person("Eve"),
    # Person("Frank"),
    # Person("Grace"),
    # Person("Hans"),
]

### Gerecht, aber nicht neidfrei (wahrscheinlich falsch)

Die Variable `estimate_variance` definiert wie gut die Personen bei ihrer Schätzung sind.

In [9]:
group1 = Group("Gerechte Aufteilung", members.copy())

estimate_variance = 0.05

while group1.cake > 0:
    
    # filter members who already have a slice
    members_filtered = [member for member in group1.members if member.position == 0]
    
    if len(members_filtered) == 2:
        m1 = members_filtered[len(members_filtered)-1]
        m2 = members_filtered[len(members_filtered)-2]
        # get estimate for last two members
        m1.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)
        m2.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)
        # find mean of estimates
        mean_estimate = (m1.estimate_percent + m2.estimate_percent) / 2
        # split remaining cake equally
        small_part = mean_estimate if mean_estimate < 0.5 else 1 - mean_estimate
        # check which member gets the small part
        # The idea is that the member with the "better" estimate gets the bigger part
        if  mean_estimate < 0.5:
            m1.percent = small_part if m1.estimate_percent < m2.estimate_percent else 1 - small_part
            m2.percent = 1 - small_part if m1.estimate_percent < m2.estimate_percent else small_part
        else :
            m1.percent = small_part if m1.estimate_percent > m2.estimate_percent else 1 - small_part
            m2.percent = 1 - small_part if m1.estimate_percent > m2.estimate_percent else small_part
        # give remaining cake to members
        m1.cake = group1.cake * m1.percent
        m2.cake = group1.cake * m2.percent
        # set position of members
        m1.position = len(group1.members) - len(members_filtered) + 1
        m2.position = len(group1.members) - len(members_filtered) + 2
        # set cake to 0
        group1.cake = 0
        
    else: 
        # get estimate for all remaining members
        for member in members_filtered:
            member.estimate_percent = 1 / len(members_filtered) + random.uniform(-estimate_variance, estimate_variance)

        # find member with smallest estimate
        member_smallest_estimate = min(members_filtered, key=lambda member: member.estimate_percent)
        # find member with second smallest estimate
        member_second_smallest_estimate = min(members_filtered, key=lambda member: member.estimate_percent if member != member_smallest_estimate else 1)
        # find mean of estimates
        mean_estimate = (member_smallest_estimate.estimate_percent + member_second_smallest_estimate.estimate_percent) / 2
        # split remaining cake one time and give it to member with smallest estimate
        member_smallest_estimate.percent = mean_estimate
        member_smallest_estimate.cake = mean_estimate * group1.cake 
        # set position of member
        member_smallest_estimate.position = len(group1.members) - len(members_filtered) + 1
        # reduce cake
        group1.cake -= member_smallest_estimate.cake
       
# print results
print(group1)

Gruppe: Gerechte Aufteilung
- Mitglieder: 4
  - 1. Aufteilung: Charlie schätzt 20.04% und bekommt 20.84 (20.84%) Kuchen (20.84%, diff 0.8)
  - 2. Aufteilung: Alice schätzt 29.26% und bekommt 25.2 (25.2%) Kuchen (31.84%, diff 2.58)
  - 3. Aufteilung: David schätzt 50.14% und bekommt 27.39 (27.39%) Kuchen (50.76%, diff 0.63)
  - 4. Aufteilung: Bob schätzt 51.39% und bekommt 26.57 (26.57%) Kuchen (49.24%, diff -2.16)


### Neidfreies Teilen

Aufgabe übersprungen.
[Quelle](https://www.spektrum.de/news/neidfreies-teilen-wie-man-einen-kuchen-fair-aufteilt/1432192)

### Exercise 2

Betrachten Sie folgendes Spiel: <br>
Eine Gruppe von N Personen stellt sich in beliebiger Ordnung auf. Jede Person bekommt dabei 1 bis max 4 Nachbarn zugewiesen (z.B. links, rechts, vorne, hinten). Nun werden für jede Person 2 Karten mit dem Namen der Person beschriftet. Eine Karte wird aus dem Spiel genommen, die restlichen Karten werden gemischt und an die Spieler so verteilt. 1 Person erhält 1 Karte, die restlichen Personen zwei. Nun geben die Spieler ihre Karten so lange an ihre Nachbarn weiter, bis am Ende alle Spieler:innen alle Karten mit ihrem Namen in Händen halten. Aber Achtung: es darf zu jedem Zeitpunkt nur eine Karte weitergegeben werden und das nur an eine:n Spieler:in mit nur einer Karte in der Hand.

1. In der vergangenen Übung haben wir die Greedy-Strategie als Lösungsstrategie kennengelernt. Was bedeutet diese Strategie für dieses Spiel? Führt sie immer zu einer Lösung?
2. Entwerfen Sie für dieses Spiel einen Algorithmus, nach dem die Karten weitergegeben werden.
3. Jeder Algorithmus hat eine Ein- und eine Ausgabe. Welches sind zulässige Eingaben für Ihren Algorithmus? Wie beschreiben Sie eine korrekte Ausgabe?
